### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Convolution2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
tensorflow.__version__

'2.8.0'

In [3]:
tensorflow.keras.__version__

'2.8.0'

### Image Data Agumentation

In [4]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'F:\\Spider\\Conversation_engine_for_deaf_and_dumb\\conversation engine-updated\\flask new\\Flask\\Dataset\\training_set',target_size=(64, 64),batch_size=32,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'F:\\Spider\\Conversation_engine_for_deaf_and_dumb\\conversation engine-updated\\flask new\\Flask\\Dataset\\test_set',target_size=(64, 64),batch_size=32,
                                            color_mode='rgb',class_mode='categorical') 

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [6]:
print(x_train.class_indices)#checking the number of classes

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}


In [7]:
print(x_test.class_indices)#checking the number of classes

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}


In [8]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 1750,
         1: 1750,
         2: 1750,
         3: 1750,
         4: 1750,
         5: 1750,
         6: 1750,
         7: 1750,
         8: 1750})

### Creating the model

In [9]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
#classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
#classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
#classifier.add(Dense(units=128,,activation='softmax'))
classifier.add(Dense(units=9, activation='softmax')) # softmax for more than 2



In [10]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 9)                 276777    
                                                                 
Total params: 277,673
Trainable params: 277,673
Non-trainable params: 0
_________________________________________________________________


### Compiling the model

In [11]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [12]:
classifier.fit_generator(x_train,steps_per_epoch =len(x_train),epochs=2, validation_data=x_test,validation_steps =len(x_test))
# No of images in test set

C:\Users\SHARAN\AppData\Local\Temp/ipykernel_18336/1807209069.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(x_train,steps_per_epoch =len(x_train),epochs=2, validation_data=x_test,validation_steps =len(x_test))


Epoch 1/2
493/493 [==============================] - 296s 592ms/step - loss: 0.2610 - accuracy: 0.9219 - val_loss: 0.2693 - val_accuracy: 0.9516
Epoch 2/2
493/493 [==============================] - 92s 186ms/step - loss: 0.0665 - accuracy: 0.9799 - val_loss: 0.3059 - val_accuracy: 0.9640


### Saving our model

In [13]:
# Save the model
classifier.save('conversation.h5')

In [14]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model("conversation.h5") #loading the model for testing

In [17]:
img = image.load_img("F:\\Spider\\Conversation_engine_for_deaf_and_dumb\\conversation engine-updated\\flask new\\Flask\\Dataset\\test_set\\A\\1.png",target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape

t=(model.predict(x))#predicting the classes
pred=(np.argmax(t,axis=1))
pred

array([0], dtype=int64)

In [18]:
index=['A', 'B', 'C','D','E','F','G','H','I']
result=str(index[pred[0]])
result

'A'